In [2]:
import sys
# sys.path.append(r"Y:\Users\Yingyue\Gates_Lab_Suite-master")
sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master")
from Core_Definition import *
from Auto_Algorithm import *
from Visualization import *
import os
from SPAM import *
import numpy as np
import xlsxwriter as xlsx
from scipy import optimize
import random
import math
from math import floor

In [26]:
def state_init_bell_pairs(qc,n):
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("HAD",i))
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("CNOT",i,int(i+n)))
        
#should we set_mapping in this function? need to think about if it takes mapping into account   
def add_pauli_twirl(qc,n):
    pauliLayer = [random.choice(['I','X','Y','Z']) for j in range(n)]
    q_index=0
    for pauli in pauliLayer :
        if (pauli=="I"):
            pass
        elif (pauli=="X"):
            qc.Add_Gate(Quantum_Gate("SKAX",q_index,angle=1))
        elif (pauli=="Y"):
            qc.Add_Gate(Quantum_Gate("SKAY",q_index,angle=1))
        elif (pauli=="Z"):
            qc.Add_Gate(Quantum_Gate("AZ",q_index,angle=1))
        q_index+=1
    return pauliLayer
    
def add_clifford_layer(qc,n,clifford):
    if (clifford=="CNOT"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("CNOT",2*i,2*i+1))
            
def bell_measurement(qc,n):
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("CNOT",i,int(i+n)))
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("HAD",i))

def map_statepop_2_ibm_mapping(counts,n):
    counts_ibm_mapping=[0 for i in range (2**n)]
    idx_ibm_mapping=[]
    counts_ibm_mapping_dic={}
    if len(str((n)))<2:
        f="00"+str(n)+"b"
    elif len(str((n)))<3:
        f="0"+str(n)+"b"
    else:
        raise ValueError("n is too big")
    for gates_idx in range (2**n):
        gates_idx_str=format(gates_idx,f)
        ibm_idx=0
        for i in range (n):
            ibm_idx+=2**(i)*int(gates_idx_str[i])
        ibm_idx_str=format(ibm_idx,f)
#         idx_ibm_mapping.append(ibm_idx_str)
        counts_ibm_mapping[ibm_idx]=counts[gates_idx]
        counts_ibm_mapping_dic[ibm_idx_str]=counts[gates_idx]
    return counts_ibm_mapping, counts_ibm_mapping_dic
        
        
    
    
    
    

In [27]:
nqubit=2
# ancillae=2
S=Quantum_Circuit(2*nqubit,"qc")

depth=[2**x for x in range(1,6)] #depth for Concatenation
C=10 #number of samples per depth
all_circuits=[]#a list of lists. circuits for all depths. each element is a list of circuit of a certain depth d. 
circuits=[] #a list of circuits with a certain depth d
eab_data=[] #a list of dictionaries. Each dictionary is the results and relevant info of one circuit
results={}
for d in depth:
    print ("\nDepth = %d"%d)
    print ("Number of samples = %d"%C)
    for i in range (C):
        S.depth=0
        S.gates=[]
        state_init_bell_pairs(S,nqubit)

        for j in range(d):
        #add pauli layer
            layer=add_pauli_twirl(S,nqubit)
            # print (layer)

            #add clifford layerr
            add_clifford_layer(S,nqubit,"CNOT")

        layer=add_pauli_twirl(S,nqubit)
        bell_measurement(S,nqubit)
#         print (S.GatesLab_Sequence())
        circuits.append(S.GatesLab_Sequence())
        results["n"]=C
        results["L"]=d
        statef=S.Simulate()
        counts=[round(2000*x) for x in statef.population]
        counts_ibm_mapping, counts_ibm_mapping_dic=map_statepop_2_ibm_mapping(counts,n)
        results["counts"]=counts_ibm_mapping_dic
        if i==0:
            print (result)
        result={}
        
    all_circuits.append(circuits)
    circuits=[]




Depth = 2
Number of samples = 10
{'n': 10, 'L': 2, 'counts': {'000': 0, '100': 0, '010': 0, '110': 0, '001': 0, '101': 0, '011': 0, '111': 0}}

Depth = 4
Number of samples = 10
{'n': 10, 'L': 4, 'counts': {'000': 0, '100': 0, '010': 0, '110': 0, '001': 2000, '101': 0, '011': 0, '111': 0}}

Depth = 8
Number of samples = 10
{'n': 10, 'L': 8, 'counts': {'000': 0, '100': 2000, '010': 0, '110': 0, '001': 0, '101': 0, '011': 0, '111': 0}}

Depth = 16
Number of samples = 10
{'n': 10, 'L': 16, 'counts': {'000': 0, '100': 0, '010': 0, '110': 0, '001': 0, '101': 0, '011': 0, '111': 0}}

Depth = 32
Number of samples = 10
{'n': 10, 'L': 32, 'counts': {'000': 0, '100': 0, '010': 0, '110': 0, '001': 0, '101': 0, '011': 0, '111': 0}}


In [ ]:
#generate emulated data and format emulated data 
#variable "results" is the formatted simulated data
#To Do: figure out how to analyze results with Senrui's code



In [19]:
a=format(14, '005b')
type(a)

str

In [21]:
n=5
if len(str((n)))<2:
    f="00"+str(n)+"b"
elif len(str((n)))<3:
    f="0"+str(n)+"b"
else:
    raise ValueError("n is too big")
print (f)
idx_str=format(0,f)
print (idx_str)

005b
00000


In [25]:
n=3
counts=[x for x in range (8)]
counts_ibm_mapping=[0 for i in range (2**n)]
idx_ibm_mapping=[]
counts_ibm_mapping_dic={}
if len(str((n)))<2:
    f="00"+str(n)+"b"
elif len(str((n)))<3:
    f="0"+str(n)+"b"
else:
    raise ValueError("n is too big")
for gates_idx in range (2**n):
    gates_idx_str=format(gates_idx,f)
    ibm_idx=0
    for i in range (n):
        ibm_idx+=2**(i)*int(gates_idx_str[i])
    ibm_idx_str=format(ibm_idx,f)
    idx_ibm_mapping.append(ibm_idx_str)
    counts_ibm_mapping[ibm_idx]=counts[gates_idx]
    counts_ibm_mapping_dic[ibm_idx_str]=counts[gates_idx]
print (idx_ibm_mapping)
print (counts_ibm_mapping)
print (counts_ibm_mapping_dic)

['000', '100', '010', '110', '001', '101', '011', '111']
[0, 4, 2, 6, 1, 5, 3, 7]
{'000': 0, '100': 1, '010': 2, '110': 3, '001': 4, '101': 5, '011': 6, '111': 7}


In [5]:
b = "Hello, World!"
print(b[-5:])

orld!
